In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [8]:
playlist = pd.read_csv("../data/result/playlist_reco_1.csv")

In [9]:
playlist

,artist,album,track_x,uri,id,artist_track,track_y,count,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_href,analysis_url,duration_ms,time_signature
0,SMOLIK / KEV FOX,SMOLIK / KEV FOX,Run,spotify:track:6tVGyszugxw0PLzBshkg7t,6tVGyszugxw0PLzBshkg7t,SMOLIK / KEV FOX: Run,NaN,0.0,NaN,0.744,0.792,7,-5.586,0,0.0435,0.411000,0.000273,0.0771,0.367,117.965,audio_features,https://api.spotify.com/v1/tracks/6tVGyszugxw0...,https://api.spotify.com/v1/audio-analysis/6tVG...,222693,4
1,The Cure,Greatest Hits,Lovesong,spotify:track:49edirvFZwWjxAMZJRg1hN,49edirvFZwWjxAMZJRg1hN,The Cure: Lovesong,NaN,0.0,NaN,0.580,0.869,9,-4.349,0,0.0297,0.375000,0.276000,0.1590,0.736,139.945,audio_features,https://api.spotify.com/v1/tracks/49edirvFZwWj...,https://api.spotify.com/v1/audio-analysis/49ed...,206907,4
2,Cecilia Krull,My Life Is Going On (Música Original De La Ser...,My Life Is Going On (Música Original De La Ser...,spotify:track:5ZEQjTroORCu6uWvZrdeNc,5ZEQjTroORCu6uWvZrdeNc,Cecilia Krull: My Life Is Going On (Música Ori...,NaN,0.0,NaN,0.427,0.647,6,-8.944,0,0.0353,0.102000,0.000002,0.1330,0.298,99.753,audio_features,https://api.spotify.com/v1/tracks/5ZEQjTroORCu...,https://api.spotify.com/v1/audio-analysis/5ZEQ...,214200,4
3,The Killers,Sam's Town,Bones,spotify:track:6TUd6BeKaEvhakhh6T2YJW,6TUd6BeKaEvhakhh6T2YJW,The Killers: Bones,NaN,0.0,NaN,0.472,0.902,9,-3.607,1,0.0643,0.000679,0.124000,0.0822,0.567,139.570,audio_features,https://api.spotify.com/v1/tracks/6TUd6BeKaEvh...,https://api.spotify.com/v1/audio-analysis/6TUd...,227000,4
4,The Killers,Sam's Town,Bones,spotify:track:4shKoD4U0rsE6UDjyPEjyK,4shKoD4U0rsE6UDjyPEjyK,The Killers: Bones,NaN,0.0,NaN,0.472,0.902,9,-3.607,1,0.0643,0.000679,0.124000,0.0822,0.567,139.570,audio_features,https://api.spotify.com/v1/tracks/4shKoD4U0rsE...,https://api.spotify.com/v1/audio-analysis/4shK...,227000,4
5,Macklemore,The Unplanned Mixtape,And We Danced (feat. Ziggy Stardust),spotify:track:1FCQEg7wOK9IIBuxx63krr,1FCQEg7wOK9IIBuxx63krr,Macklemore: And We Danced (feat. Ziggy Stardust),NaN,0.0,NaN,0.758,0.677,5,-6.598,1,0.0782,0.456000,0.000000,0.0954,0.713,124.998,audio_features,https://api.spotify.com/v1/tracks/1FCQEg7wOK9I...,https://api.spotify.com/v1/audio-analysis/1FCQ...,246213,4
6,U2,Songs Of Innocence,The Troubles - Alternative Version,spotify:track:0XKClTokiXeQ6XTv4WxWJk,0XKClTokiXeQ6XTv4WxWJk,U2: The Troubles - Alternative Version,NaN,0.0,NaN,0.409,0.634,6,-9.432,1,0.0373,0.207000,0.051900,0.1260,0.375,150.014,audio_features,https://api.spotify.com/v1/tracks/0XKClTokiXeQ...,https://api.spotify.com/v1/audio-analysis/0XKC...,272305,4
7,Queen,Forever,Love Kills - The Ballad,spotify:track:2MAdGSEc0bs15KTuXPax0G,2MAdGSEc0bs15KTuXPax0G,Queen: Love Kills - The Ballad,NaN,0.0,NaN,0.496,0.792,7,-6.134,0,0.0418,0.115000,0.000026,0.1300,0.327,126.585,audio_features,https://api.spotify.com/v1/tracks/2MAdGSEc0bs1...,https://api.spotify.com/v1/audio-analysis/2MAd...,252613,4
8,Skunk Anansie,What You Do For Love,What You Do For Love,spotify:track:4rC6R0Dhe9LavndPwAt2B0,4rC6R0Dhe9LavndPwAt2B0,Skunk Anansie: What You Do For Love,NaN,0.0,NaN,0.598,0.689,8,-6.519,1,0.0637,0.000402,0.000004,0.1420,0.523,142.033,audio_features,https://api.spotify.com/v1/tracks/4rC6R0Dhe9La...,https://api.spotify.com/v1/audio-analysis/4rC6...,200934,4
9,Malky,Where Is Piemont,Theodore,spotify:track:6Mq87hGdnLjM6nVQYoB98P,6Mq87hGdnLjM6nVQYoB98P,Malky: Theodore,NaN,0.0,NaN,0.516,0.503,4,-10.748,0,0.0429,0.110000,0.000002,0.1640,0.205,96.832,audio_features,https://api.spotify.com/v1/tracks/6Mq87hGdnLjM...,https://api.spotify.com/v1/audio-analysis/6Mq8...,213507,4


## Create the corresponding Playlist in Spotify

- `playlist_add_items`
- `user_playlist_create`

In [5]:
GDPR_DATA_FOLDER = "../data/raw/MyData/"
client_id_path = "../secrets/client_id.txt"
client_secret_path = "../secrets/client_secret.txt"
user_id_path = "../secrets/user_id.txt"

# TODO: put everything into a single json file

with open(client_id_path, "r") as file:
    client_id = file.read().strip("\n")
    
with open(client_secret_path, "r") as file:
    client_secret = file.read().strip("\n")
    
with open(user_id_path, "r") as file:
    user_id = file.read().strip("\n")

In [7]:
scope = "playlist-modify-public"

In [10]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://example.com",
                                               scope=scope))

In [13]:
playlist_name = "Liked Recommendations - the next"
description = "Let's go for it"

In [14]:
res = sp.user_playlist_create(user=user_id, name=playlist_name, description=description)

In [15]:
playlist_id = res["id"]

In [16]:
sp.playlist_add_items(playlist_id, playlist["uri"].tolist())

{'snapshot_id': 'MywwZGEyMzM0YWIyNDUyNTNjN2JiNmQxNTE1OTM2MDI1ZDZmYjQ4YWI4'}